In [ ]:
import gzip
import zipfile
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# 设置天河二号远程服务器上的目标路径
remote_zip_file_path = '/GPUFS/sysu_htlin_6/tester01.zip.gz'
server_target_dir = '/GPUFS/sysu_htlin_6/'

# 创建本地数据文件夹
if not os.path.exists(server_target_dir):
    os.makedirs(server_target_dir)

zipgz_folder_path = '/GPUFS/sysu_htlin_6/tester01.zip.gz'
with gzip.open(zipgz_folder_path, 'rb') as gz_file:
    with zipfile.ZipFile(gz_file, 'r') as zip_file:
        zip_file.extractall(server_target_dir)

# 设置数据路径
train_datagen = ImageDataGenerator(rescale=1.0/255, validation_split=0.15)
train_generator = train_datagen.flow_from_directory(os.path.join(server_target_dir, 'tester01'),
                                                    target_size=(224, 224),
                                                    batch_size=32,
                                                    class_mode='categorical',
                                                    subset='training')

# 构建模型
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')  
])

# 编译模型
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',  
              metrics=['accuracy'])

# 训练模型
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    epochs=10,
                    validation_data=train_generator,
                    validation_steps=train_generator.samples // train_generator.batch_size)

# 评估模型
validation_generator = train_datagen.flow_from_directory(os.path.join(server_target_dir, 'tester01'),
                                                         target_size=(224, 224),
                                                         batch_size=32,
                                                         class_mode='categorical',
                                                         subset='validation')
test_loss, test_acc = model.evaluate(validation_generator, steps=validation_generator.samples // validation_generator.batch_size)
print("Validation accuracy:", test_acc)

# 设置模型保存路径
model_save_path = '/GPUFS/sysu_htlin_6/model/model.h5'

# 保存模型为HDF5文件格式
model.save(model_save_path)

# 打印保存路径
print("Model saved to:", model_save_path)